### Imports

First we need to correctly set the python environment. This is done by adding the top directory of the repository to the Python path. Once that's done, we can import various packages from inside the repository.

In [ ]:
import sys, yaml, math, numpy as np, torch
sys.path.append('/scratch') # This line is equivalent to doing source scripts/source_me.sh in a bash terminal
from torch.utils.data import DataLoader
from SparseBase.trainers import SparseTrainer
from SparseBase import utils
from SparseNOvA import datasets

### Configuring

Most of the training options are set in a configuration YAML file. We're going to load this config, and then the options inside will be passed to the relevent piece of the training framework.

In [ ]:
with open('/scratch/SparseNOvA/config/sparse_nova.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

### Loading inputs

Here we load the dataset and the trainer, which is responsible for building the model and overseeing training. There's a block of code which is responsible for slicing the full dataset up into a training dataset and a validation dataset.

In [ ]:
full_dataset = datasets.get_dataset(**config['data'])
trainer = SparseTrainer(**config['trainer'])

fulllen = len(full_dataset)
tv_num = math.ceil(fulllen*config['data']['t_v_split'])
splits = np.cumsum([fulllen-tv_num,0,tv_num])
collate = utils.collate_sparse_minkowski_2stack

train_dataset = torch.utils.data.Subset(full_dataset,np.arange(start=0,stop=splits[0]))
valid_dataset = torch.utils.data.Subset(full_dataset,np.arange(start=splits[1],stop=splits[2]))
train_loader = DataLoader(train_dataset, collate_fn=collate, **config['data_loader'], shuffle=True)
valid_loader = DataLoader(valid_dataset, collate_fn=collate, **config['data_loader'], shuffle=False)

### Building the model

The trainer will load the network architecture and compile it into a model

In [ ]:
trainer.build_model(**config['model'])

### Training!

Once all the setup is done, all that's left is to run training and save some summary statistics to file.

In [ ]:
train_summary = trainer.train(train_loader, valid_data_loader=valid_loader, **config['trainer'])
print(train_summary)
torch.save(train_summary, 'summary_test.pt')